In [57]:
import pandas as pd
import numpy as np
from utilities import Data_cleaning

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.express.colors import sample_colorscale
from IPython.display import clear_output

import math
import copy
%matplotlib inline

pd.set_option('display.max_rows', 500)

In [58]:
df_demographic = pd.read_hdf('../data/flattened_database_merged_with_session_executions_v03_(andrea).h5',  key='data')
df_demographic.index.names = ["session_execution_id"]
df_demographic.replace(np.nan, 0, inplace=True)

df = pd.read_hdf('../data/filtered_dataset_v1.h5', key='data')
df_demographic.shape, df.shape

((51355, 9190), (46024, 8677))

In [59]:
# bmi calculation
df_demographic_filtered = copy.copy(df_demographic)
df_demographic_filtered['users_bmi'] = df_demographic['users_weight'] / (df_demographic['users_height'] / 100) ** 2

df_demographic_columns = ['user_programs_user_id',
                            'users_created_at',
                            'users_date_of_birth', 
                            'users_activity_level', 
                            'users_gender', 
                            'users_body_type', 
                            'users_weight', 
                            'users_height', 
                            'users_body_fat', 
                            'users_goal', 
                            'users_bmi',
                            'session_executions_difficulty_feedback',
                            'session_executions_enjoyment_feedback',
                            'programs_pro', 
                            'programs_name_en', 
                            'programs_description_en', 
                            'users_last_sign_in_at', 
                            'user_programs_active',
                            'user_programs_completed',
                            'users_newsletter_subscription']

df_demographic_filtered = df_demographic_filtered[df_demographic_columns]

In [60]:
# merge demographic data with the main dataset by index

df_merged = df.merge(df_demographic_filtered,
                                  how ='inner',
                                  left_index = True,
                                  right_index= True)

# Rename user_programs_user_id
df_merged = df_merged.rename(columns={'user_programs_user_id': 'user_programs_user_id'})

In [61]:
df_users = df_merged.drop_duplicates(subset=['user_programs_user_id'], keep='last')

KeyError: Index(['user_programs_user_id'], dtype='object')

In [ ]:
#user_columns = ['user_programs_user_id',
#           'users_created_at',
#           'session_executions_updated_at',
#           'users_date_of_birth', 
#           'users_activity_level', 
#           'users_gender', 
#           'users_body_type', 'users_weight', 'users_height', 'users_body_fat', 'users_goal', 'users_bmi']

# Append session_executions_updated_at to the df_demographic_columns
df_demographic_columns.append('session_executions_updated_at')

df_users = df_users[df_demographic_columns]

# reassing the index to the user_programs_user_id

df_users.set_index('user_programs_user_id', inplace=True)

# rename the columns
df_users = df_users.rename(columns={'session_executions_updated_at': 'Last_training_date'})

In [ ]:
# calculate the first training date for each user
first_training_date = df_merged.drop_duplicates(subset=['user_programs_user_id'], keep='first')[['session_executions_updated_at', 'user_programs_user_id']]

# Reassign the index to the user_programs_user_id
first_training_date.set_index('user_programs_user_id', inplace=True)

# merge the first training date with the user data
df_users = df_users.merge(first_training_date, how='inner', left_index=True, right_index=True)

df_users = df_users.rename(columns={'session_executions_updated_at': 'first_training_date'})

In [ ]:
# add the number of sessions to the user dataframe
df_users['number_of_sessions'] = df_merged['user_programs_user_id'].value_counts()

In [ ]:
# add Duration of “activity” (time from session 1 to last session)

# Sum of time of all sessions by user
duration_of_activity = df_merged.groupby('user_programs_user_id')['sum_of_time'].sum()

# Merge the duration of activity with the user dataframe
df_users = df_users.merge(duration_of_activity,
                          how ='inner',
                          left_index = True,
                          right_index= True)

# Rename the columns
df_users.rename(columns={'sum_of_time_x': 'duration_of_activity', 'sum_of_time_y': 'duration_of_sessions'}, inplace=True)


In [ ]:
# Add the duration of sessions

# calculate the duration of sessions
duration_of_sessions = df_merged.groupby('user_programs_user_id')['sum_of_time'].agg(list)

# merge the duration of sessions with the user dataframe
df_users = df_users.merge(duration_of_sessions,
                          how ='inner',
                          left_index = True,
                          right_index= True)


In [ ]:
# get month of the sessions
df_merged['month_of_sessions'] = df_merged['session_executions_updated_at'].dt.month

# get the list of the months of the sessions
month_of_sessions = df_merged.groupby('user_programs_user_id')['month_of_sessions'].agg(list)

# rename month of sessions from numbers to names

months = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}

month_of_sessions = month_of_sessions.apply(lambda x: [months[i] for i in x])

# merge the month of sessions with the user dataframe
df_users = df_users.merge(month_of_sessions,
                          how ='inner',
                          left_index = True,
                          right_index= True)

In [ ]:
# session_executions_difficulty_feedback and session_executions_enjoyment_feedback

# get the mean of the session_executions_difficulty_feedback and session_executions_enjoyment_feedback
session_executions_difficulty_feedback = df_merged.groupby('user_programs_user_id')['session_executions_difficulty_feedback'].mean()

# merge the session_executions_difficulty_feedback with the user dataframe
df_users = df_users.merge(session_executions_difficulty_feedback,
                          how ='inner',
                          left_index = True,
                          right_index= True)

# get the list of the session_executions_difficulty_feedback
session_executions_difficulty_feedback_list = df_merged.groupby('user_programs_user_id')['session_executions_difficulty_feedback'].agg(list)

# merge the session_executions_difficulty_feedback with the user dataframe
df_users = df_users.merge(session_executions_difficulty_feedback_list,
                          how ='inner',
                          left_index = True,
                          right_index= True)

# get the mean of the session_executions_enjoyment_feedback
session_executions_enjoyment_feedback = df_merged.groupby('user_programs_user_id')['session_executions_enjoyment_feedback'].mean()

# merge the session_executions_enjoyment_feedback with the user dataframe
df_users = df_users.merge(session_executions_enjoyment_feedback,
                          how ='inner',
                          left_index = True,
                          right_index= True)

# get the list of the session_executions_enjoyment_feedback
session_executions_enjoyment_feedback_list = df_merged.groupby('user_programs_user_id')['session_executions_enjoyment_feedback'].agg(list)

# merge the session_executions_enjoyment_feedback with the user dataframe
df_users = df_users.merge(session_executions_enjoyment_feedback_list,
                          how ='inner',
                          left_index = True,
                          right_index= True)

In [ ]:
# rename columns
df_users.rename(columns={'session_executions_difficulty_feedback_x': 'mean_difficulty_feedback', 
                        'session_executions_difficulty_feedback_y': 'difficulty_feedback', 
                        'session_executions_enjoyment_feedback_x': 'mean_enjoyment_feedback', 
                        'session_executions_enjoyment_feedback_y': 'enjoyment_feedback'}, inplace=True)


In [ ]:
# Calculate the duration from the first session to the last session exucuted

df_users['duration_from_first_to_last_session'] = df_users['Last_training_date'] - df_users['first_training_date']

In [ ]:
# Small test to corroborate that all the users have at least one training session

'''
df2 = pd.read_hdf('../data/filtered_dataset_v1.h5', key='data')
df2.drop(columns=['session_executions_updated_at', 'session_executions_updated_at', 'user_programs_user_id', 'date', 'session_executions_summary_total_kcal', 'session_executions_summary_effort', 'session_executions_summary_points', 'session_executions_summary_value_of_session'], inplace=True)

# count the zero values for a rows
df2['zero_values'] = df2.apply(lambda x: x.value_counts().get(0, 0), axis=1)
print(df2['zero_values'].value_counts().keys().sort_values(ascending=False))
print(df2.shape)
'''
# In the precious prints we can observe that all the rows have at least 5 values, which means that at least one training session was executed.

"\ndf2 = pd.read_hdf('../data/filtered_dataset_v1.h5', key='data')\ndf2.drop(columns=['session_executions_updated_at', 'session_executions_updated_at', 'user_programs_user_id', 'date', 'session_executions_summary_total_kcal', 'session_executions_summary_effort', 'session_executions_summary_points', 'session_executions_summary_value_of_session'], inplace=True)\n\n# count the zero values for a rows\ndf2['zero_values'] = df2.apply(lambda x: x.value_counts().get(0, 0), axis=1)\nprint(df2['zero_values'].value_counts().keys().sort_values(ascending=False))\nprint(df2.shape)\n"

In [ ]:
# Calculate the sum of time for each exercise excluding the rest time

df_merged['sum_of_time_excluding_rest_time'] = df_merged['sum_of_time'] - df_merged['Rest_sum_of_time_per_exercise']

# Calculate the mean of the sum of time for each exercise excluding the rest time

sum_of_time_excluding_rest_time = df_merged.groupby('user_programs_user_id')['sum_of_time_excluding_rest_time'].mean()

# merge the sum of time excluding rest time with the user dataframe

df_users = df_users.merge(sum_of_time_excluding_rest_time,
                          how ='inner',
                          left_index = True,
                          right_index= True)

# Calculate the list of the sum of time for each exercise excluding the rest time

sum_of_time_excluding_rest_time_list = df_merged.groupby('user_programs_user_id')['sum_of_time_excluding_rest_time'].agg(list)

# merge the sum of time excluding rest time with the user dataframe

df_users = df_users.merge(sum_of_time_excluding_rest_time_list,
                          how ='inner',
                          left_index = True,
                          right_index= True)

# rename columns
df_users.rename(columns={'sum_of_time_excluding_rest_time_x': 'mean_sum_of_time_excluding_rest_time', 
                        'sum_of_time_excluding_rest_time_y': 'sum_of_time_excluding_rest_time_list'}, inplace=True)


C:\Users\eVida-PC\AppData\Local\Temp\ipykernel_34852\4275649835.py:22: FutureWarning: Passing 'suffixes' which cause duplicate columns {'sum_of_time_excluding_rest_time_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_users = df_users.merge(sum_of_time_excluding_rest_time_list,


In [166]:
# Add number of weeks of activity

dates_by_user = df_merged.groupby('user_programs_user_id_x')['session_executions_updated_at'].agg(list)
df_dates_by_user = dates_by_user.to_frame().reset_index()

def get_weeks(dates):
    
    all_weeks = {}
    all_months = {}
    for row in dates.iterrows():       

        start_date = row[1]['session_executions_updated_at'][0]
        week_counts = {}
        month_counts = {}
        for date in row[1]['session_executions_updated_at']:
            week_number = (date - start_date).days // 7
            month_number = (date - start_date).days // 30
            week_key = f'week_{week_number}'
            month_key = f'month_{month_number}'

            # count the number of sessions per week
            if week_key in week_counts:
                week_counts[week_key] += 1
            else:
                week_counts[week_key] = 1

            # count the number of sessions per month
            if month_key in month_counts:
                month_counts[month_key] += 1
            else:
                month_counts[month_key] = 1
        
        # append the week counts to the all weeks dictionary
        all_weeks[row[1]['user_programs_user_id_x']] = week_counts
        all_months[row[1]['user_programs_user_id_x']] = month_counts
        
    return pd.Series(all_weeks), pd.Series(all_months)

# get the weeks of the sessions
weeks_frequency, months_frequency = get_weeks(df_dates_by_user)

# add a name to the weeks pandas series
weeks_frequency.name = 'weeks_frequency'

# add a name to the months pandas series
months_frequency.name = 'months_frequency'

# merge the weeks of the sessions with the user dataframe
df_users = df_users.merge(weeks_frequency,
                          how ='inner',
                          left_index = True,
                          right_index= True)

# merge the months of the sessions with the user dataframe
df_users = df_users.merge(months_frequency,
                          how ='inner',
                          left_index = True,
                          right_index= True)


In [169]:
df_users

,users_created_at,users_date_of_birth,users_activity_level,users_gender,users_body_type,users_weight,users_height,users_body_fat,users_goal,users_bmi,...,session_executions_difficulty_feedback,enjoyment_feedback,session_executions_enjoyment_feedback,duration_from_first_to_last_session,mean_sum_of_time_excluding_rest_time,sum_of_time_excluding_rest_time_list,mean_sum_of_time_excluding_rest_time,sum_of_time_excluding_rest_time_list,weeks_frequency,months_frequency
user_programs_user_id,,,,,,,,,,,,,,,,,,,,,
108,2020-11-21 11:02:22.366198,1984-08-29,2,False,0,68.000000,164.00,25.0,0,25.282570,...,"[2.0, 3.0, 7.0, 2.0, 5.0, 5.0, 5.0, 5.0]",3.0,"[3.0, 3.0, 4.0, 2.0, 3.0, 3.0, 3.0, 3.0]",302 days 16:17:04.973158,1278.75,"[709.0, 1260.0, 2372.0, 908.0, 2060.0, 909.0, ...",1278.75,"[709.0, 1260.0, 2372.0, 908.0, 2060.0, 909.0, ...","{'week_0': 1, 'week_20': 1, 'week_25': 2, 'wee...","{'month_0': 1, 'month_4': 1, 'month_6': 2, 'mo..."
112,2020-11-22 22:24:45.117192,1978-02-18,2,True,0,73.000000,178.00,20.0,0,23.040020,...,"[4.0, 6.0]",3.0,"[2.0, 4.0]",90 days 22:29:04.250765,243.00,"[50.0, 436.0]",243.00,"[50.0, 436.0]","{'week_0': 1, 'week_12': 1}","{'month_0': 1, 'month_3': 1}"
144,2020-11-30 15:05:58.467008,2020-11-30,1,True,0,60.000000,160.00,25.0,0,23.437500,...,"[0.0, 0.0]",0.0,"[0.0, 0.0]",3 days 00:57:14.939427,16.00,"[9.0, 23.0]",16.00,"[9.0, 23.0]",{'week_0': 2},{'month_0': 2}
172,2020-12-07 16:42:33.327819,1960-12-31,1,True,1,92.000000,179.00,45.0,0,28.713211,...,"[5.0, 6.0, 6.0, 4.0, 5.0]",2.8,"[2.0, 3.0, 4.0, 3.0, 2.0]",12 days 20:25:21.952077,773.20,"[944.0, 1023.0, 618.0, 956.0, 325.0]",773.20,"[944.0, 1023.0, 618.0, 956.0, 325.0]","{'week_0': 3, 'week_1': 2}",{'month_0': 5}
182,2020-12-11 11:34:00.850343,2000-01-01,1,True,0,60.000000,160.00,15.0,0,23.437500,...,[0.0],0.0,[0.0],0 days 00:00:00,132.00,[132.0],132.00,[132.0],{'week_0': 1},{'month_0': 1}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18127,2022-05-24 13:08:58.940710,1975-06-07,1,False,0,58.000000,160.00,20.0,1,22.656250,...,[6.0],5.0,[5.0],0 days 00:00:00,1310.00,[1310.0],1310.00,[1310.0],{'week_0': 1},{'month_0': 1}
18147,2022-05-25 07:12:53.616277,1980-09-20,2,False,0,55.000000,149.00,20.0,0,24.773659,...,[3.0],3.0,[3.0],0 days 00:00:00,400.00,[400.0],400.00,[400.0],{'week_0': 1},{'month_0': 1}
18157,2022-05-25 16:36:21.636963,2001-06-21,2,True,2,98.000000,172.00,20.0,0,33.126014,...,[5.0],3.0,[3.0],0 days 00:00:00,739.00,[739.0],739.00,[739.0],{'week_0': 1},{'month_0': 1}
